In [5]:
import pickle
import sys
sys.path.append("../src")
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers.convolutional import Conv2D
from nnclib.utils import model_dic

layer_stats = pickle.load(open('../layer_stats.pickle', 'rb'))
for k, i in model_dic.items():
    print(k, len(i))
model = model_dic['xception'][0]()

xception 2
vgg16 2
vgg19 2
resnet50 2
inceptionv3 2
inceptionresnetv2 2
mobilenet 2
densenet121 2
densenet169 2
densenet201 2
nasnetmobile 2
nasnetlarge 2


In [6]:
def proc_weights(w):
    print(np.shape(w))
    x, y, inch, outch = np.shape(w)
    p = np.reshape(w, (x * y * outch, inch))
    #p = p.T
    print(np.shape(p))
    a = np.argsort(p, axis=0)
    p = np.take_along_axis(p, a, axis=0)
    norm = np.linalg.norm(p, axis=0)
    p /= norm[np.newaxis, :]
    plt.plot(p)
    plt.show()


def proc_conv2d(model):
    clayers = filter(lambda t: isinstance(t, Conv2D), model.layers)
    # for clayer in list(clayers)[:5]:
    for clayer in clayers:
        print("clayer {}".format(type(clayer)))
        cweights = clayer.get_weights()[0]
        proc_weights(cweights)

In [11]:
aw = model.weights
al = model.layers
len(aw), len(al)

(236, 134)

In [17]:
al[:5]

In [20]:
list(map(lambda L: [np.shape(L.get_weights()), type(L)], al[:10]))

[[(0,), keras.engine.input_layer.InputLayer],
 [(1, 3, 3, 3, 32), keras.layers.convolutional.Conv2D],
 [(4, 32), keras.layers.normalization.BatchNormalization],
 [(0,), keras.layers.core.Activation],
 [(1, 3, 3, 32, 64), keras.layers.convolutional.Conv2D],
 [(4, 64), keras.layers.normalization.BatchNormalization],
 [(0,), keras.layers.core.Activation],
 [(2,), keras.layers.convolutional.SeparableConv2D],
 [(4, 128), keras.layers.normalization.BatchNormalization],
 [(0,), keras.layers.core.Activation]]

In [19]:
aw[:20]

[<tf.Variable 'block1_conv1_1/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'block1_conv1_bn_1/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'block1_conv1_bn_1/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'block1_conv2_1/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'block1_conv2_bn_1/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'block1_conv2_bn_1/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv1_1/depthwise_kernel:0' shape=(3, 3, 64, 1) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv1_1/pointwise_kernel:0' shape=(1, 1, 64, 128) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv1_bn_1/gamma:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv1_bn_1/beta:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv2_1/depthwise_kernel:0' shape=(3, 3, 128, 1) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv2_1/pointwise_kernel:0' shape=(1, 1, 128, 128) dtype=float32_ref>,
 <tf.Variabl

In [33]:
def prod(array):
    p = 1
    for e in array:
        p *= e
    return p

In [41]:
def magic(L):
    print(type(L))
    W = L.get_weights()
    shape = np.shape(W)
    print(len(shape), shape)
    if len(shape) > 1:
        print('MAGIC')
        y = shape[-2]
        x = prod(list(shape[:-2])) * shape[-1]
        np.reshape(W, [x, y])
        print(x, y)

for L in al:
    magic(L)

<class 'keras.engine.input_layer.InputLayer'>
1 (0,)
<class 'keras.layers.convolutional.Conv2D'>
5 (1, 3, 3, 3, 32)
MAGIC
288 3
<class 'keras.layers.normalization.BatchNormalization'>
2 (4, 32)
MAGIC
32 4
<class 'keras.layers.core.Activation'>
1 (0,)
<class 'keras.layers.convolutional.Conv2D'>
5 (1, 3, 3, 32, 64)
MAGIC
576 32
<class 'keras.layers.normalization.BatchNormalization'>
2 (4, 64)
MAGIC
64 4
<class 'keras.layers.core.Activation'>
1 (0,)
<class 'keras.layers.convolutional.SeparableConv2D'>
1 (2,)
<class 'keras.layers.normalization.BatchNormalization'>
2 (4, 128)
MAGIC
128 4
<class 'keras.layers.core.Activation'>
1 (0,)
<class 'keras.layers.convolutional.SeparableConv2D'>
1 (2,)
<class 'keras.layers.normalization.BatchNormalization'>
2 (4, 128)
MAGIC
128 4
<class 'keras.layers.convolutional.Conv2D'>
5 (1, 1, 1, 64, 128)
MAGIC
128 64
<class 'keras.layers.pooling.MaxPooling2D'>
1 (0,)
<class 'keras.layers.normalization.BatchNormalization'>
2 (4, 128)
MAGIC
128 4
<class 'keras.lay